<a href="https://www.kaggle.com/code/hanpat99/bitcoin-price-movement-using-coingecko-api?scriptVersionId=116621645" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Bitcoin price movement using CoinGecko API V3**

We will see how Bitcoin price moved in the last 1 year and perform various analyses therafter. We will use the **[CoinGecko API](https://www.coingecko.com/en/api/documentation).** to get the price data for the 60 days with 24 observation per day, 1 per hour. We will find the ***max, min, open***, and ***close price*** per ***day***, meaning we will have 365 candlesticks and use this data to generate the candlestick graph. Although we are using the CoinGecko API we will use a Python client/wrapper for the API called PyCoinGecko. PyCoinGecko will make performing the requests easy and it will deal with the endpoint targeting.

Here's a description of a candlestick.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/explanation.png" alt="Candle Stick Definition" width="800" height="600">

##  **But first, lemme install a package!**

In [1]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance
!pip install --upgrade nbformat

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 833.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 824.0 kB/s eta 0:00:00
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.4.0
    Uninstalling nbformat-5.4.0:
      Successfully uninstalled nbformat-5.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.


## Let's get down to business

### 1. "Get" the data from the API!

First, we obtain the data from **CoinGecko** using the get request, **get_coin_market_chart_by_id(id, vs_currency, days**). **id** is the name of the coin you want, **vs_currency** is the currency you want the price in, and **days** is how many days back from today you want. 

We will store the data in dict **bitcoin_data**.

In [2]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days= 365)

In [3]:
type(bitcoin_data )

#Uncomment below line and execute to see the data pulled
#bitcoin_data

bitcoin_data.keys()

dict_keys(['prices', 'market_caps', 'total_volumes'])

### 2. Extract the list **prices** from the dictionary
The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.

In [4]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1642464000000, 42298.341116872325],
 [1642550400000, 42395.45879157343],
 [1642636800000, 41749.55143098559],
 [1642723200000, 40707.6824143097],
 [1642809600000, 36508.799789830824]]

### 3. Lets turn the **prices** data into a Pandas DataFrame

In [5]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])

data.head()

,TimeStamp,Price
0,1642464000000,42298.341117
1,1642550400000,42395.458792
2,1642636800000,41749.551431
3,1642723200000,40707.682414
4,1642809600000,36508.799790


### 4. Convert the timestamp to a readable format

Now that we have the DataFrame we will convert the **timestamp** to **datetime** and save it as a column called **date**. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime.

In [6]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

### 5. Use GROUPBY on the cleaned prices data to obtain the final results
Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

In [7]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})
candlestick_data

date         Price                                          
                          min           max         first          last
0    2022-01-18  42298.341117  42298.341117  42298.341117  42298.341117
1    2022-01-19  42395.458792  42395.458792  42395.458792  42395.458792
2    2022-01-20  41749.551431  41749.551431  41749.551431  41749.551431
3    2022-01-21  40707.682414  40707.682414  40707.682414  40707.682414
4    2022-01-22  36508.799790  36508.799790  36508.799790  36508.799790
..          ...           ...           ...           ...           ...
360  2023-01-13  18866.810331  18866.810331  18866.810331  18866.810331
361  2023-01-14  19941.780543  19941.780543  19941.780543  19941.780543
362  2023-01-15  21019.206614  21019.206614  21019.206614  21019.206614
363  2023-01-16  20853.230569  20853.230569  20853.230569  20853.230569
364  2023-01-17  21175.337737  21259.180550  21175.337737  21259.180550

[365 rows x 5 columns]

### 6. Build a Viz!
Finally we are now ready to use **plotly** to create our Candlestick Chart.

In [8]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=True)

fig.show()

###Insights Insights Insights!

This analysis was conducted on 17 Jan 2022 for a time-period of 365 days preceding current day.

1. The value of Bitcoin has fluctuated wildly.

    A. The closing maximum occured on Mar 30, 2022 and the closing      minimum occured on Nov 10, 2022.
    
    B. The range of value is 31252.44
    
2. The strong price fluctuation indicates a high volatility, and by extension, a high risk.

3. Cryptocurrency such as Bitcoin may prove to be extremely profitable if invested in carefully. While there seems to be a slight upwards trend on the current day, this is only marginal when compared to the steep fall it experienced over the year.


### Yay! We have built a candlestick chart for the last 60 days. You may also repeat this analysis for any number of days by updating the value passed to the parameter ****days**** in the ****get request****.



### We will be performing more interesting analyses and fun visualizations with this data in the upcoming weeks. Stay tuned!
